In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog

from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models.distillation import TrayColumn
from idaes.generic_models.unit_models.distillation.condenser \
    import CondenserType, TemperatureSpec

In [2]:
from idaes.generic_models.properties.core.examples.BT_ideal \
    import configuration

In [3]:
# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) 

# Add properties parameter blocks to the flowsheet with specifications
m.fs.props = GenericParameterBlock(default=configuration)

In [4]:
m.fs.unit = TrayColumn(default={
                       "number_of_trays": 10,
                       "feed_tray_location": 5,
                       "condenser_type": CondenserType.partialCondenser,
                       "condenser_temperature_spec":
                           TemperatureSpec.customTemperature,
                       "property_package": m.fs.props,
                       "has_heat_transfer": False,
                       "has_pressure_change": False})

# m.fs.unit = TrayColumn(default={
#                        "number_of_trays": 10,
#                        "feed_tray_location": 5,
#                        "condenser_type": CondenserType.totalCondenser,
#                        "condenser_temperature_spec":
#                            TemperatureSpec.atBubblePoint,
#                        "property_package": m.fs.props,
#                        "has_heat_transfer": False,
#                        "has_pressure_change": False})

In [5]:
m.fs.unit.feed.flow_mol.fix(40)
m.fs.unit.feed.temperature.fix(368)
m.fs.unit.feed.pressure.fix(101325)
m.fs.unit.feed.mole_frac_comp[0, "benzene"].fix(0.5)
m.fs.unit.feed.mole_frac_comp[0, "toluene"].fix(0.5)

In [6]:
m.fs.unit.condenser.reflux_ratio.fix(1.4)
m.fs.unit.condenser.condenser_pressure.fix(101325)

m.fs.unit.reboiler.boilup_ratio.fix(1.3)

# m.fs.unit.condenser.distillate.temperature[0].fix(298.15)

m.fs.unit.condenser.vapor_outlet.temperature.fix(368)

In [7]:
print(degrees_of_freedom(m))
print(degrees_of_freedom(m.fs.unit.condenser))
print(degrees_of_freedom(m.fs.unit.reboiler))

0
5
5


In [8]:
m.fs.unit.initialize() 

2021-04-01 10:42:18 [INFO] idaes.init.fs.unit: Begin initialization.
2021-04-01 10:42:18 [WARNING] idaes.init.fs.unit: Solver not provided. Default solver(ipopt) being used for initialization.
2021-04-01 10:42:18 [INFO] idaes.init.fs.unit.feed_tray: Begin initialization.
2021-04-01 10:42:18 [WARNING] idaes.init.fs.unit.feed_tray: Solver not provided. Default solver(ipopt)  being used for initialization.
2021-04-01 10:42:18 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Starting initialization
2021-04-01 10:42:18 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:18 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Equilibrium temperature initialization completed.
2021-04-01 10:42:18 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:18 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: 

2021-04-01 10:42:20 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:20 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Equilibrium temperature initialization completed.
2021-04-01 10:42:20 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:20 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Property initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:20 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Starting initialization
2021-04-01 10:42:20 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:20 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Equilibrium temperature initializati

2021-04-01 10:42:22 [INFO] idaes.init.fs.unit.rectification_section[4]: Begin initialization.
2021-04-01 10:42:22 [WARNING] idaes.init.fs.unit.rectification_section[4]: Solver not provided. Default solver(ipopt)  being used for initialization.
2021-04-01 10:42:22 [INFO] idaes.init.fs.unit.rectification_section[4].properties_in_liq: Starting initialization
2021-04-01 10:42:22 [INFO] idaes.init.fs.unit.rectification_section[4].properties_in_liq: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:22 [INFO] idaes.init.fs.unit.rectification_section[4].properties_in_liq: Equilibrium temperature initialization completed.
2021-04-01 10:42:22 [INFO] idaes.init.fs.unit.rectification_section[4].properties_in_liq: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:22 [INFO] idaes.init.fs.unit.rectification_section[4].properties_in_liq: Property initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:22 [INFO] idaes.i

2021-04-01 10:42:24 [INFO] idaes.init.fs.unit.stripping_section[7].properties_out: Property initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:24 [INFO] idaes.init.fs.unit.stripping_section[7].properties_out: Property package initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:24 [INFO] idaes.init.fs.unit.stripping_section[7]: Mass balance solve optimal - Optimal Solution Found.
2021-04-01 10:42:24 [INFO] idaes.init.fs.unit.stripping_section[7]: Mass and energy balance solve optimal - Optimal Solution Found.
2021-04-01 10:42:24 [INFO] idaes.init.fs.unit.stripping_section[7]: Mass, energy and pressure balance solve optimal - Optimal Solution Found.
2021-04-01 10:42:24 [INFO] idaes.init.fs.unit.stripping_section[7]: Initialization complete, status optimal - Optimal Solution Found.
2021-04-01 10:42:24 [INFO] idaes.init.fs.unit.stripping_section[8]: Begin initialization.
2021-04-01 10:42:24 [WARNING] idaes.init.fs.unit.stripping_section[8]: Solver not provide

2021-04-01 10:42:27 [INFO] idaes.init.fs.unit.stripping_section[10].properties_in_vap: Property initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:27 [INFO] idaes.init.fs.unit.stripping_section[10].properties_out: Starting initialization
2021-04-01 10:42:27 [INFO] idaes.init.fs.unit.stripping_section[10].properties_out: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:27 [INFO] idaes.init.fs.unit.stripping_section[10].properties_out: Equilibrium temperature initialization completed.
2021-04-01 10:42:27 [INFO] idaes.init.fs.unit.stripping_section[10].properties_out: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:27 [INFO] idaes.init.fs.unit.stripping_section[10].properties_out: Property initialization: optimal - Optimal Solution Found.
2021-04-01 10:42:27 [INFO] idaes.init.fs.unit.stripping_section[10].properties_out: Property package initialization: optimal - Optimal Solution Found.
2021-04-01 

In [9]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [10]:
results = solver.solve(m)

status = solver.solve(m, tee = True)

Ipopt 3.13.2: tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. S

In [11]:
m.display()

Model unknown

  Variables:
    None

  Objectives:
    None

  Constraints:
    None

  Blocks:
    Block fs
    
      Variables:
        None
    
      Objectives:
        None
    
      Constraints:
        None
    
      Blocks:
        Block fs.props
        
          Variables:
            None
        
          Objectives:
            None
        
          Constraints:
            None
        
          Blocks:
            Block fs.props.Liq
            
              Variables:
                None
            
              Objectives:
                None
            
              Constraints:
                None
            Block fs.props.Vap
            
              Variables:
                None
            
              Objectives:
                None
            
              Constraints:
                None
            Block fs.props.benzene
            
              Variables:
                pressure_crit : Size=1, Index=None
                    Key

                        Key            : Lower : Body                    : Upper
                        ('Vap', 'Liq') :   0.0 : -1.0186340659856796e-10 :   0.0
                    eq_mole_frac_tbub : Size=2
                        Key                       : Lower : Body                   : Upper
                        ('Vap', 'Liq', 'benzene') :   0.0 :  8.731149137020111e-11 :   0.0
                        ('Vap', 'Liq', 'toluene') :   0.0 : 1.4551915228366852e-11 :   0.0
                    _teq_constraint_Vap_Liq : Size=1
                        Key  : Lower : Body                   : Upper
                        None :   0.0 : -5.684341886080802e-14 :   0.0
                    eq_temperature_dew : Size=1
                        Key            : Lower : Body : Upper
                        ('Vap', 'Liq') :   0.0 :  0.0 :   0.0
                    eq_mole_frac_tdew : Size=2
                        Key                       : Lower : Body                   : Upper
               

                        Key  : Lower  : Value              : Upper : Fixed : Stale : Domain
                        None : 273.15 : 368.22735069731004 :   450 : False : False : NonNegativeReals
                    flow_mol_phase : Phase molar flow rates
                        Size=2, Index=fs.props.phase_list
                        Key : Lower : Value                 : Upper : Fixed : Stale : Domain
                        Liq :     0 : 0.0010764882962543386 :  1000 : False : False : NonNegativeReals
                        Vap :     0 :     30.31873875798592 :  1000 : False : False : NonNegativeReals
                    mole_frac_phase_comp : Phase mole fractions
                        Size=4, Index=fs.props._phase_component_set
                        Key                : Lower : Value               : Upper : Fixed : Stale : Domain
                        ('Liq', 'benzene') :     0 :   0.404898589450429 :  None : False : False :  Reals
                        ('Liq', 'toluene') : 

                        Key            : Lower  : Value              : Upper : Fixed : Stale : Domain
                        ('Vap', 'Liq') : 273.15 : 361.85057712150143 :   450 : False : False :  Reals
                    _mole_frac_tbub : Vapor mole fractions at bubble temperature
                        Size=2, Index=fs.unit.rectification_section[4].properties_in_vap[0.0]._mole_frac_tbub_index
                        Key                       : Lower : Value               : Upper : Fixed : Stale : Domain
                        ('Vap', 'Liq', 'benzene') :     0 :  0.8091179753585849 :  None : False : False :  Reals
                        ('Vap', 'Liq', 'toluene') :     0 : 0.19088202464141515 :  None : False : False :  Reals
                    temperature_dew : Dew point temperature
                        Size=1, Index=fs.props._pe_pairs
                        Key            : Lower  : Value              : Upper : Fixed : Stale : Domain
                        ('Vap', 'Liq') : 

                    0.0 :   0.0 :  0.0 :   0.0
                pressure_drop_equation : Size=1
                    Key : Lower : Body : Upper
                    0.0 :   0.0 :  0.0 :   0.0
            
              Blocks:
                Block fs.unit.feed_tray.properties_in_feed[0.0]
                
                  Variables:
                    flow_mol :  Total molar flowrate
                        Size=1, Index=None
                        Key  : Lower : Value : Upper : Fixed : Stale : Domain
                        None :     0 :    40 :  1000 :  True :  True : NonNegativeReals
                    mole_frac_comp : Mixture mole fractions
                        Size=2, Index=fs.props.component_list
                        Key     : Lower : Value : Upper : Fixed : Stale : Domain
                        benzene :     0 :   0.5 :  None :  True :  True :  Reals
                        toluene :     0 :   0.5 :  None :  True :  True :  Reals
                    pressure : State pr

...

                    mole_frac_phase_comp : Phase mole fractions
                        Size=4, Index=fs.props._phase_component_set
                        Key                : Lower : Value               : Upper : Fixed : Stale : Domain
                        ('Liq', 'benzene') :     0 :  0.2573240166431006 :  None : False : False :  Reals
                        ('Liq', 'toluene') :     0 :  0.7426759833568993 :  None : False : False :  Reals
                        ('Vap', 'benzene') :     0 : 0.45694766720017754 :  None : False : False :  Reals
                        ('Vap', 'toluene') :     0 :  0.5430523327998223 :  None : False : False :  Reals
                    phase_frac : Phase fractions
                        Size=2, Index=fs.props.phase_list
                        Key : Lower : Value                  : Upper : Fixed : Stale : Domain
                        Liq :     0 : 4.6445484121718754e-05 :  None : False : False :  Reals
                        Vap :     0 :     0

                        Size=1, Index=None
                        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
                        None :  None : 375.74016391642454 :  None : False : False :  Reals
                    temperature_bubble : Bubble point temperature
                        Size=1, Index=fs.props._pe_pairs
                        Key            : Lower  : Value              : Upper : Fixed : Stale : Domain
                        ('Vap', 'Liq') : 273.15 : 369.64223014892804 :   450 : False : False :  Reals
                    _mole_frac_tbub : Vapor mole fractions at bubble temperature
                        Size=2, Index=fs.unit.stripping_section[9].properties_in_vap[0.0]._mole_frac_tbub_index
                        Key                       : Lower : Value              : Upper : Fixed : Stale : Domain
                        ('Vap', 'Liq', 'benzene') :     0 : 0.5812142502395984 :  None : False : False :  Reals
                        ('Vap',

                        Size=4, Index=fs.props._phase_component_set
                        Key                : Lower : Value               : Upper : Fixed : Stale : Domain
                        ('Liq', 'benzene') :     0 : 0.18993471751813137 :  None : False : False :  Reals
                        ('Liq', 'toluene') :     0 :  0.8100652824818686 :  None : False : False :  Reals
                        ('Vap', 'benzene') :     0 : 0.36094220665106314 :  None : False : False :  Reals
                        ('Vap', 'toluene') :     0 :  0.6390577933489369 :  None : False : False :  Reals
                    phase_frac : Phase fractions
                        Size=2, Index=fs.props.phase_list
                        Key : Lower : Value              : Upper : Fixed : Stale : Domain
                        Liq :     0 : 0.6332161863667937 :  None : False : False :  Reals
                        Vap :     0 : 0.3667838136332063 :  None : False : False :  Reals
                    _teq 

                            ('Vap', 'Liq', 'benzene') :   0.0 : -1.4551915228366852e-11 :   0.0
                            ('Vap', 'Liq', 'toluene') :   0.0 :  -2.546585164964199e-11 :   0.0
                        _teq_constraint_Vap_Liq : Size=1
                            Key  : Lower : Body : Upper
                            None :   0.0 :  0.0 :   0.0
                        eq_temperature_dew : Size=1
                            Key            : Lower : Body                  : Upper
                            ('Vap', 'Liq') :   0.0 : 4.440892098500626e-16 :   0.0
                        eq_mole_frac_tdew : Size=2
                            Key                       : Lower : Body                    : Upper
                            ('Vap', 'Liq', 'benzene') :   0.0 : -2.9103830456733704e-11 :   0.0
                            ('Vap', 'Liq', 'toluene') :   0.0 :  -7.275957614183426e-12 :   0.0
                        equilibrium_constraint : Size=2
                          

                mole_frac_comp_equality : Size=2
                    Key              : Lower : Body : Upper
                    (0.0, 'benzene') :   0.0 :  0.0 :   0.0
                    (0.0, 'toluene') :   0.0 :  0.0 :   0.0
                pressure_equality : Size=1
                    Key : Lower : Body : Upper
                    0.0 :   0.0 :  0.0 :   0.0
                temperature_equality : Size=1
                    Key : Lower : Body : Upper
                    0.0 :   0.0 :  0.0 :   0.0
            Block fs.unit.feed_liq_out_expanded
            
              Variables:
                None
            
              Objectives:
                None
            
              Constraints:
                flow_mol_equality : Size=1
                    Key : Lower : Body : Upper
                    0.0 :   0.0 :  0.0 :   0.0
                mole_frac_comp_equality : Size=2
                    Key              : Lower : Body : Upper
                    (0.0, 'benzene') :   

In [12]:
# m.pprint()

In [13]:
# m.fs.unit.report()

ConfigurationError: Unit model fs.unit does not have the standard Port names (inet and outlet). Please contact the unit model developer to develop a unit specific stream table.